### The following code snippet 
+ combines physiological episodes with relevant RR interval information
+ divides each episode's duration into 5-min windows with 1-min sliding window; retains windows with at least 4 or more distinct minutes
+ computes various aggregated HRV features from the 5-min windows for each episode

In [1]:
from cerebralcortex import Kernel
from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
from math import ceil
import pyspark.sql.functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, ArrayType, \
TimestampType, IntegerType, DateType
from scipy.stats import iqr
import warnings

pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings("ignore")
CC = Kernel("/home/jupyter/cc3_moods_conf/", study_name="moods")
print('PySpark Version :'+CC.sparkContext.version)

PySpark Version :3.1.2


### Fetch, clean and filter episodes data

In [2]:
import pickle
with open('/home/jupyter/sneupane/MOODS/analysis/papers/CHI/dataframe/final_df.pickle', 'rb') as handle:
    final_df = pickle.load(handle)
df_users_episodes = final_df
df_users_episodes.columns = df_users_episodes.columns.str.lower()

In [3]:
cols = ['stress_id', 'starttime', 'endtime', 'value', 'episode_class', 'deleted',
         'state', 'user_generated', 'user_rating', 'location', 'selected', 'created',
         'last_updated', 'label', 'stressor', 'version', 'user']

In [4]:
uuid_substitutions={"4e3c01a1-4f61-3329-b843-edd72eaece63":"62c32dbd-a32d-3ecd-a9f1-fb9bc40fff66",
                   "37686d8b-d47b-33e9-99b1-496196e7ada2":"ff54abe7-a4dd-3c10-a00d-e9bcc68ee92b",
                   "68d89413-5a7f-3a3b-9e6c-b22d2699307a":"07c8b674-2c13-3a33-a973-cf1cab70d9f9",
                   "7e0aa5f7-96cd-3a95-a28d-e3ef3684e0e1":"71e88740-be67-382a-a21f-78fba469cb13",
                   "559c51b0-672e-32d9-a1d7-f7b36b5190af":"f7400971-a4b7-326a-8120-2e1db7f91cca",
                   "b9b13911-dda5-38de-bb0d-becd0c9a4c7d":"835c291b-ecf4-32da-9a58-67bfe5ecfb7f",
                   "fd1574fe-3093-3519-949e-15c98b4bc73a":"d8404d54-51d1-35fc-9bf7-32aa5942c575",
                   "336482c0-71cf-39c8-96bb-8aeb7ffa4d3f":"b0a58353-edf9-3213-9bd5-808e7ad42bd5"}

In [5]:
def data_quality_issues(pdf, 
                        uuid_substitutions):
    filt = pdf['starttime'] > pdf['endtime']
    pdf.loc[filt, 'starttime'], pdf.loc[filt, 'endtime'] = (pdf.loc[filt, 'endtime'], 
                                                            pdf.loc[filt, 'starttime'])
    pdf['user'] = pdf['user'].replace(uuid_substitutions)
    return pdf
     
def tweak_ds_users_episodes(df_users_episodes, 
                            cols):        
    return (df_users_episodes[cols]
            .assign(starttime=pd.to_datetime(df_users_episodes['starttime']), 
                    endtime=pd.to_datetime(df_users_episodes['endtime']),
                    eps_dur=((df_users_episodes['endtime'] - df_users_episodes['starttime'])
                             .dt.total_seconds() / 60),
                    date=df_users_episodes['starttime'].dt.date, 
                    rated=np.where(df_users_episodes['user_rating'].notnull(), 1, 0))
            .sort_values(by=['user', 'endtime'])
           )

df_users_episodes = data_quality_issues(tweak_ds_users_episodes(df_users_episodes, 
                                                                cols), 
                                        uuid_substitutions)

In [6]:
users = list(df_users_episodes['user'].unique())

### Extract annotated episodes

In [7]:
check_eps_pdf = (df_users_episodes.loc[df_users_episodes['user']
                                       .isin(users)]
                 .loc[lambda df: df['rated'] == 1])
len(check_eps_pdf)

26732

In [8]:
check_eps_pdf = check_eps_pdf[['user', 'stress_id', 'starttime', 'endtime']]
                                                                       
check_eps_sdf = CC.sparkSession.createDataFrame(check_eps_pdf)
check_eps_sdf.show(5)

+--------------------+---------+-------------------+-------------------+
|                user|stress_id|          starttime|            endtime|
+--------------------+---------+-------------------+-------------------+
|00222a15-7274-34b...|   162009|2022-09-20 14:56:18|2022-09-20 15:05:28|
|00222a15-7274-34b...|   162201|2022-09-20 15:41:22|2022-09-20 16:19:26|
|00222a15-7274-34b...|   162202|2022-09-20 16:19:26|2022-09-20 16:29:27|
|00222a15-7274-34b...|   162210|2022-09-20 17:55:30|2022-09-20 18:08:30|
|00222a15-7274-34b...|   162539|2022-09-21 14:18:10|2022-09-21 14:31:11|
+--------------------+---------+-------------------+-------------------+
only showing top 5 rows



### Extract RR-interval data

In [9]:
def get_rr_intervals(stream_name, CC):
    ds = CC.get_stream(stream_name, version="all")
    return ds

CC = Kernel("/home/jupyter/cc3_moods_conf/", study_name="moods")
stream_name = 'rr_interval--org.md2k.watch--fossil_watch_sport'
ds_rr = (get_rr_intervals(stream_name, CC)
         .data
         .orderBy(['user', 'localtime'])
         .withColumn('date', F.col("localtime").cast('date'))
         .withColumn('HR', 60000/F.col('interval')))

In [22]:
# ds_rr.show(5, truncate=False)

### Filter RR intervals of users whose annoated episodes were extracted in the previous step 

In [11]:
check_rr_sdf = ds_rr.filter(ds_rr['user'].isin(users))

In [12]:
check_rr_sdf.select(['user']).distinct().count()

120

### Merge episodes with relevant RR information

In [23]:
join_eps_rr_sdf = (check_rr_sdf.join(check_eps_sdf)
                   .filter((check_rr_sdf['user'] == check_eps_sdf['user']) &
                           (check_rr_sdf['localtime'] >= check_eps_sdf['starttime']) &
                           (check_rr_sdf['localtime'] <= check_eps_sdf['endtime']))
                   .drop(check_eps_sdf['user']).orderBy(['stress_id', 'localtime']))

In [ ]:
# join_eps_rr_sdf.show(5)

### Create 5-min windows with 1-min sliding window; keep windows with at least 4 or more distinct minutes

In [17]:
from datetime import datetime, timedelta

return_schema = StructType([StructField("user", StringType()),
                            StructField("stress_id", IntegerType()),
                            StructField("localtime_st", TimestampType()),
                            StructField("localtime_et", TimestampType()),
                            StructField("date", DateType()),
                            StructField("interval_list", ArrayType(DoubleType())),
                            StructField("rank", IntegerType()),
                            StructField("distinct_min_count", IntegerType())                                        
                           ])
                                
@pandas_udf(return_schema, PandasUDFType.GROUPED_MAP)
def create_window(eps_df):
    window_dur = 300
    sliding_dur = 60
    eps_df = eps_df.sort_values(by=['localtime'])
    if (eps_df['endtime'].iloc[0] - eps_df['starttime'].iloc[0]).total_seconds() < window_dur:
        return pd.DataFrame(columns=[f.name for f in return_schema])

    rank = 0        
    win_st = eps_df['starttime'].iloc[0]
    win_et = win_st + timedelta(seconds=window_dur)
    windows = []
    while win_et <= eps_df['endtime'].iloc[-1]:
        lt_list = eps_df.loc[(eps_df['localtime'] >= win_st) 
                             & (eps_df['localtime'] <= win_et), 'localtime'].to_list()

        unique_ts = {pd.to_datetime(ts).minute for ts in lt_list}
        if len(unique_ts) >= 4:
            rr_list = eps_df.loc[(eps_df['localtime'] >= win_st) 
                                 & (eps_df['localtime'] <= win_et), 'interval'].to_list()

            windows.append([eps_df['user'].iloc[0], eps_df['stress_id'].iloc[0], 
                            win_st, win_et, 
                            eps_df['date'].iloc[0], 
                            rr_list, rank, len(unique_ts)])
            rank += 1
        win_st = win_st + timedelta(seconds=sliding_dur)
        win_et = win_st + timedelta(seconds=window_dur)
        
    return pd.DataFrame(windows, columns=[f.name for f in return_schema])

In [18]:
valid_5_min_window = join_eps_rr_sdf.groupBy(['user', 'stress_id']).apply(create_window)

In [24]:
# valid_5_min_window.orderBy('stress_id').show(5)

### Compute aggregated HRV features from the 5-min windows

In [20]:
return_schema = StructType([StructField("user", StringType()),
                            StructField("stress_id", IntegerType()),
                            StructField('hrv_mean', DoubleType()),
                            StructField('hrv_median', DoubleType()),
                            StructField('hrv_iqr', DoubleType()),
                            StructField('hrv_80P', DoubleType()),
                            StructField('hrv_20P', DoubleType()),
                            StructField('SDANN', DoubleType()),
                            StructField('RMSSD', DoubleType()),
                           ])
                                
@pandas_udf(return_schema, PandasUDFType.GROUPED_MAP)
def window_features(user_data):
    result_df = pd.DataFrame(columns=[c.name for c in return_schema])    
    group_dfs = user_data.groupby('stress_id')
    for stress_id, df in group_dfs:
        df['avg_IBI'] = df['interval_list'].apply(lambda x : np.mean(x))
        idx = len(result_df)
        result_df.loc[idx, "user"] = df['user'].iloc[0]
        result_df.loc[idx, "stress_id"] = stress_id
        result_df.loc[idx, 'hrv_mean'] = np.mean(df['avg_IBI'])
        result_df.loc[idx, 'hrv_median'] = np.median(df['avg_IBI'])
        result_df.loc[idx, 'hrv_iqr'] = iqr(df['avg_IBI'])        
        result_df.loc[idx, 'hrv_80P'] = np.percentile(df['avg_IBI'], 80)
        result_df.loc[idx, 'hrv_20P'] = np.percentile(df['avg_IBI'], 20)
        result_df.loc[idx, 'SDANN'] = np.std(df['avg_IBI'])
        result_df.loc[idx, 'RMSSD'] = np.sqrt(np.mean(np.square(np.diff(df['avg_IBI']))))
    return result_df

In [32]:
features_sdf = (valid_5_min_window
                .orderBy(["stress_id", "localtime_st"])
                .groupBy('user').apply(window_features))

In [26]:
features_sdf.select(['stress_id', 'hrv_mean', 'hrv_median', 
                     'hrv_iqr', 'hrv_80P', 'hrv_20P', 'SDANN', 'RMSSD']).show(5)

+---------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+
|stress_id|         hrv_mean|       hrv_median|           hrv_iqr|          hrv_80P|          hrv_20P|             SDANN|             RMSSD|
+---------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+
|   189816|788.6829134275097|795.3780864197531| 33.43076333277975|808.6480922135969|767.7766497082149|21.455545589884377|  9.47965117349114|
|   189818|731.2042933311726|729.3590504451039| 25.88859530004413|744.2334090909092|715.5899586758094|20.003336898839038|17.489572353460037|
|   189821|746.7963322994786|740.0636604774536| 4.838740509396416|750.0378333587737|738.1612560488113|14.714896100462914|16.647170185749015|
|   189824|832.0487012987013|832.0487012987013|12.656655844155807|839.6426948051948| 824.454707792208|12.656655844155807|25.313311688311614|
|   189826|76

### Normalize features individual wise

In [33]:
@pandas_udf(return_schema, PandasUDFType.GROUPED_MAP)
def normalize_window_features(user_feature_data):
    for col in user_feature_data.columns:
        if col in ["user", "stress_id"]:continue
        user_feature_data[col] = ((user_feature_data[col] - user_feature_data[col].mean()) 
                                  / user_feature_data[col].std(ddof=0))
    return user_feature_data

In [34]:
normalized_features_sdf = features_sdf.groupBy('user').apply(normalize_window_features)
normalized_features_pdf = normalized_features_sdf.toPandas()

In [35]:
normalized_features_pdf[['stress_id', 'hrv_mean', 'hrv_median', 
                     'hrv_iqr', 'hrv_80P', 'hrv_20P', 'SDANN', 'RMSSD']].head()

,stress_id,hrv_mean,hrv_median,hrv_iqr,hrv_80P,hrv_20P,SDANN,RMSSD
0,189816,0.980,1.107,0.873,1.208,0.686,0.754,-0.582
1,189818,-0.312,-0.320,0.461,-0.259,-0.401,0.623,0.333
2,189821,0.039,-0.088,-0.689,-0.127,0.069,0.146,0.237
3,189824,1.955,1.900,-0.262,1.914,1.866,-0.039,1.226
4,189826,0.404,0.504,0.165,0.564,0.280,0.579,-0.267
